In [1]:
from lxml import html
import requests
import pandas as pd

In [2]:
local_authority_address = '/cambridge-e07000008'
base_url = 'http://www.ukcensusdata.com'
page = requests.get(base_url + local_authority_address)
tree = html.fromstring(page.content)

In [3]:
links = tree.xpath('/html[1]/body[1]/div[2]/div[1]/div[1]/div[3]/p[1]/a/@href')
ward_links = [base_url + x for x in links]
names = tree.xpath('/html[1]/body[1]/div[2]/div[1]/div[1]/div[3]/p[1]/a/text()')
ward_links = {k: v for k, v in zip(names, ward_links)}
ward_links

{'Abbey': 'http://www.ukcensusdata.com/abbey-e05002702',
 'Arbury': 'http://www.ukcensusdata.com/arbury-e05002703',
 'Castle': 'http://www.ukcensusdata.com/castle-e05002704',
 'Cherry Hinton': 'http://www.ukcensusdata.com/cherry-hinton-e05002705',
 'Coleridge': 'http://www.ukcensusdata.com/coleridge-e05002706',
 'East Chesterton': 'http://www.ukcensusdata.com/east-chesterton-e05002707',
 "King's Hedges": 'http://www.ukcensusdata.com/kings-hedges-e05002708',
 'Market': 'http://www.ukcensusdata.com/market-e05002709',
 'Newnham': 'http://www.ukcensusdata.com/newnham-e05002710',
 'Petersfield': 'http://www.ukcensusdata.com/petersfield-e05002711',
 "Queen Edith's": 'http://www.ukcensusdata.com/queen-ediths-e05002712',
 'Romsey': 'http://www.ukcensusdata.com/romsey-e05002713',
 'Trumpington': 'http://www.ukcensusdata.com/trumpington-e05002714',
 'West Chesterton': 'http://www.ukcensusdata.com/west-chesterton-e05002715'}

In [4]:
ward_pages = {}
for ward, link in ward_links.items():
    page = requests.get(link)
    ward_pages[ward] = page

In [5]:
ward_stats = {}
for ward, page in ward_pages.items():
    tree = html.fromstring(page.content)
    labels = tree.xpath("//div[@id='content']//div//td//strong//text()")
    labels = [x.split(':')[0] for x in labels]
    values = tree.xpath("//div[@id='content']//tr//td[2]/text()")
    values = [float(x.replace(',', '')) for x in values]
    stats = {k:v for k, v in zip(labels, values)}
    ward_stats[ward] = stats

In [6]:
ward_df = pd.DataFrame.from_dict(ward_stats)

In [7]:
ward_df.to_json('ward_stats.json')
pd.read_json('ward_stats.json')

,Abbey,Arbury,Castle,Cherry Hinton,Coleridge,East Chesterton,King's Hedges,Market,Newnham,Petersfield,Queen Edith's,Romsey,Trumpington,West Chesterton
All Residents,9907.0,9070.0,9785.0,8780.0,9386.0,9405.0,9142.0,7150.0,7867.0,8333.0,9127.0,9252.0,8034.0,8629.0
Area (hectares),395.0,149.0,341.0,368.0,193.0,261.0,157.0,169.0,445.0,105.0,452.0,149.0,733.0,153.0
Average household size,2.4,2.3,2.2,2.4,2.4,2.3,2.3,2.0,2.3,2.2,2.4,2.4,2.1,2.2
Number of households,4106.0,3886.0,2082.0,3676.0,3788.0,4038.0,3918.0,1661.0,1711.0,3366.0,3424.0,3857.0,3472.0,3729.0
Population density (people per hectare),25.1,60.7,28.7,23.8,48.6,36.0,58.2,42.2,17.7,79.1,20.2,62.2,11.0,56.6
Residents in communal living,46.0,154.0,5256.0,0.0,409.0,166.0,150.0,3778.0,4004.0,769.0,827.0,46.0,619.0,298.0
Residents in households,9861.0,8916.0,4529.0,8780.0,8977.0,9239.0,8992.0,3372.0,3863.0,7564.0,8300.0,9206.0,7415.0,8331.0
